In [1]:
from mssm.models import *
from mssm.src.python.compare import compare_CDL
from mssm.src.python.gamm_solvers import deriv_transform_mu_eta,deriv_transform_eta_beta
import pandas as pd
import numpy as np
import scipy as scp
from mssmViz.plot import *
from numdifftools import Gradient,Hessian
import matplotlib.pyplot as plt

######################################### Setup #########################################

# Define general smooth family using only llk info:
class LBFGSGENSMOOTHFamily(GENSMOOTHFamily):
    # Implementation of the ``GENSMOOTHFamily`` class to estimate a general smooth model.

    # References:
    #    - Wood, Pya, & Säfken (2016). Smoothing Parameter and Model Selection for General Smooth Models.
    #    - Nocedal & Wright (2006). Numerical Optimization. Springer New York.
    

    def __init__(self, pars: int, links:[Link], llkfun:Callable, *llkargs) -> None:
        super().__init__(pars, links, *llkargs)
        self.llkfun = llkfun
    
    def llk(self, coef, coef_split_idx, y, Xs):
        return self.llkfun(coef, coef_split_idx, self.links, y, Xs,*self.llkargs)
    
    def gradient(self, coef, coef_split_idx, y, Xs):
        """
        Function to evaluate gradient for gamm model.
        """
        coef = coef.reshape(-1,1)
        split_coef = np.split(coef,coef_split_idx)
        eta_mu = Xs[0]@split_coef[0]
        eta_sd = Xs[1]@split_coef[1]
        
        # Set the Gamlss family
        gammlss_family = GAUMLSS([self.links[0],self.links[1]])

        d1eta,d2eta,d2meta = deriv_transform_mu_eta(y,[self.links[0].fi(eta_mu),self.links[1].fi(eta_sd)],gammlss_family)
        grad,_ = deriv_transform_eta_beta(d1eta,d2eta,d2meta,Xs)
        #print(pgrad.flatten())
        return grad
    
    def hessian(self, coef, coef_split_idx, y, Xs):
        return None
    


# GAMM Analysis

The first part of the practical focuses on estimating Generalized additive models with ``mssm``.

With ``mssm`` we can estimate a broad variety of Generalized additive (mixed) models as they are also supported in ``mgcv`` (Wood, 2017). Smooths that are available in ``mgcv`` and are also supported here include regular univariate smooths, binary smooth terms, Tensor smooth interactions, by-factor smooths, and random non-linear smooths. ``mssm`` also supports random effects (random intercepts, random slopes, and as mentioned random smooths).

Note, that ``mssm`` by default uses a different basis and optimizer compared to ``gam`` and ``bam``. Models here are always estimated to optimize (approximately) the REML criterion. For more details, you can take a lookt at the documentation, hosted [here](https://jokra1.github.io/mssm/index.html)! The ``mssm`` tutorial available [here](https://github.com/JoKra1/mssm_tutorials) also contains useful information.

## Data

Below we load in reaction time data from the lexical decision task experiment described [here](https://doi.org/10.1162/jocn_a_02214).
We will focus here on an analysis of the RT data. RTs were collected under three stimulus conditions (Words, Pseudo-words, random strings).
We will look here at the effect of one or two continuous variables on the RTs: log(OLD20) and log(Google frequency scores).
These are available in the columns ``logOLD`` and ``logGoogleFreq`` respectively. We also have variables for the specific
stimuli (``stims_var``) shown on individual trials, the trial order identifier (``trials_var``), and the subject identifier (``subjects_var``).


The data used to create the specific dataframe is available on [Github](https://github.com/JoKra1/HSMM_LD_EEG) - but since some of the
packages needed to form the dataframe are no longer available on CRAN I completed these steps for you and you only have to load in the
final result.

In [ ]:
# Import the data
dat = pd.read_csv("rt_dat.csv")

# mssm requires that the data-type for variables used as factors is 'O'=object
dat = dat.astype({'subjects_var': 'O',
                  'blocks_var':'O',
                  'conditions':'O',
                  'stims_var':'O'})

dat.head()

## Exercise 1

Consult the [mssm tutorial](https://github.com/JoKra1/mssm_tutorials) and find the table that maps ``mgcv`` formulas to ``mssm`` formulas. Then translate the ``mgcv`` formula ``rts_var ~ conditions + s(logGoogleFreq,by=conditions) + s(logGoogleFreq,subjects_var,bs="fs")`` into an instance of ``Formula``. You can also check the documentation for ``Formula`` (and ``lhs``) [here](https://jokra1.github.io/mssm/mssm.src.python.html#module-mssm.src.python.formula).

In [3]:
# Your code goes here.
formula = None

## Exercise 2

Consult the [mssm tutorial](https://github.com/JoKra1/mssm_tutorials) or the [documentation of the GAMM class](https://jokra1.github.io/mssm/mssm.html#mssm.models.GAMM) to implement a Gaussian additive model for the ``formula`` you specified above. Then call ``model.fit()``

In [5]:
# Your code goes here.
model = None

## Exercise 3

Consult the [mssm tutorial](https://github.com/JoKra1/mssm_tutorials) to learn how to visualize the estimated partial effects of frequency on the mean. Use the ``plot`` function from the ``mssmViz`` package to visualize these effects. What can you learn about the effect of frequency on RTs - does it differ between conditions?

In [7]:
# Your code goes here.

## Exercise 4

This question focuses on model validation. Consult the [mssm tutorial](https://github.com/JoKra1/mssm_tutorials) to learn how to create residual/validation plots. Make sure to also extract the residuals from the fitted model (see the [documentation](https://jokra1.github.io/mssm/mssm.html#mssm.models.GAMM) for the ``GAMM`` class) and to visualize how they change over the time-course of the experiment (hint: consider the ``trials_var`` variable. hint 2: carefully consider whether any ``NANs`` were present and thus removed. See the [documentation](https://jokra1.github.io/mssm/mssm.src.python.html#mssm.src.python.formula.Formula) for the ``Formula`` class). Is the model appropriate? How can it be improved?

In [9]:
# Your code goes here.

## Exercise 5

Create a second formula that also includes a random smooth of ``trials_var``(i.e., in ``mgcv`` this would be ``s(trials_var,subjects_var,bs='fs')``). What does this term capture? Then specify a second model - this time assuming either a ``Gamma`` or ``Inverse Gaussian`` family (with ``log`` link). Consult the [documentation on families](https://jokra1.github.io/mssm/mssm.src.python.html#module-mssm.src.python.exp_fam) to see how these are implemented in ``mssm``. Then, fit the model, plot the partial effects of frequency (important: one what scale are the plots visualized by default? Hint: change the ``response_scale`` argument of the plot function), and inspect the residuals (Which residuals are visualized in the plots? Try re-creating them using the ``model2.get_resid()`` method - see [here](https://jokra1.github.io/mssm/mssm.html#mssm.models.GAMM.get_resid) for the documentation.).

In [11]:
# Your code goes here.
formula2 = None
model2 = None

## Exercise 6

This exercise focuses on model selection. Fit a third model, which includes a smooth of ``logOLD`` as well. Use the ``model3.print_smooth_terms()`` ([documentation](https://jokra1.github.io/mssm/mssm.html#mssm.models.GAMM.print_smooth_terms)) method and the ``compare_CDL()`` ([documentation](https://jokra1.github.io/mssm/mssm.src.python.html#mssm.src.python.compare.compare_CDL)) function to determine whether the more complex model offers a better fit to the data.

In [14]:
# Your code goes here
formula3 = None
model3 = None

# General Smooth Models

We will now turn towards estimating more general smooth models. However, we will make our life a bit easier - we will continue to estimate GAMMs (or GAMMLSS) but act like we don't know how to do that and can only provide a log-likelihood function that we would like to optimize (as would be the case for a more general smooth model). This allows us to compare the estimates to the "real" GAMM estimates obtained so far.

We start with considering the log-likelihood of a GAMM. To be able to compute the latter, we need to revisit the assumptions underlying a GAMM. In brief, we assume that each of our $N$ observations $y_i$ is a realization from a random variable $Y_i \sim \mathcal{E}(\mu_i,\phi)$ with mean $\mu_i$ and scale parameter $\phi$ (for a Gaussian additive model we assume that $\mathcal{E} = N$ and $\phi=\sigma$). A function $g(\mu_i)$ of $\mu_i$ - the mean of the RVs - is modeled as an additive combination of smooth functions and parametric terms (the "additive model part"). That is, $g(\mu_i)=\eta_i$, where $\eta_i$ is the model prediction for observation $y_i$. In contrast to $\mu_i$ which can be different for every $Y_i$, $\phi$ is assumed to be shared by all RVs!

We further assume that all the $Y_i$ are mutually independent, because that makes computing the log-lieklihood easier (**Note**: the independence assumption is not **necessary** for more **general** smooth models). Specifically, if $p_{Y_i}(y_i|\mu_i,\phi)$ denotes the probability-density for observation $y_i$ given $\mathcal{E}$, $\mu_i$, and $\phi$, the log-likelihood is simply:

$\sum_i^N log(p_{Y_i}(y_i|\mu_i,\phi))$.

Note, that $\mu_i = g^{-1}(\mathbf{X}_i\boldsymbol{\beta}) = g^{-1}(\eta_i)$, where $\mathbf{X}_i$ denotes row $i$ of model matrix $\mathbf{X}$ and $\boldsymbol{\beta}$ holds all the weights/coefficients implied by the model of the mean. Further, note that we can also represent (some function of) $\phi$ as a product of a second model matrix and a second coefficient vector! Specifically, we can write that $\phi_i = g_\phi^{-1}(\mathbf{X}_{\phi,i}\boldsymbol{\beta}_\phi)$.

For a GAMM $\phi_1 = \phi_2 ... = \phi_N=\phi$, so $\mathbf{X}_{\phi,i}=1$ and $\boldsymbol{\beta}_\phi$ will contain only a single coefficient and this re-formulation will look rather inefficient. However, when we express it like this we can immediately provide more complex models for $\phi_i$ - allowing it to differ between $Y_i$ as well.

## Exercise 7

Provided with the likelihood computations outlined above, implement the function below that compute the log-likelihood of a Gaussian GAMM. The function takes 5 arguments. ``coef`` is a ``numpy.array``, in this case holding $[\boldsymbol{\beta},\boldsymbol{\beta}_\phi]$. ``coef_split_idx`` is a list that can be used to split ``coef``, in this case into $\boldsymbol{\beta}$ and $\boldsymbol{\beta}_\phi$:

```
coef = coef.reshape(-1,1)
split_coef = np.split(coef,coef_split_idx)
```

``split_coef`` will then be a list, holding $\boldsymbol{\beta}$ (at index 0) and $\boldsymbol{\beta}_\phi$ at index (1). ``links`` is a list holding classes that in this case implement $g$ and $g_\phi$ (see the [documentation](https://jokra1.github.io/mssm/mssm.src.python.html#mssm.src.python.exp_fam.Link) for the ``Link`` class.). ``y`` is a ``numpy.array`` holding the $N$ observations. ``Xs`` is a list of model matrices, in this case holding $\mathbf{X}$ (at index 0) and $\mathbf{X}_\phi$ (at index 1). You can use these last three arguments to compute both, the vector holding all means $\mu_i$ and the vector holding all $\phi_i$ (which as mentioned before will be identical for a GAMM):

```
eta_mu = Xs[0]@split_coef[0]
eta_sd = Xs[1]@split_coef[1]

mu = links[0].fi(eta_mu) # vector of means
sd = links[1].fi(eta_sd) # vector of scales
```

Given ``y``, ``mu`` and ``sd`` (both ``numpy.array``s) you can then compute the log-densities $log(p_{Y_i}(y_i|\mu_i,\phi))$ (using for example ``scipy.stats.norm.logpdf()``, see [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html)). The sum of those is the log-likelihood and should be returned.

**Note**: You don't have to worry about specifying values for any of the arguments, ``mssm`` will arrange that for you.

In [3]:
def llk_gamm_fun(coef,coef_split_idx,links,y,Xs):
    # Likelihood for a Gaussian GAMM - implemented so
    # that the model can be estimated using the general smooth model class.
    coef = coef.reshape(-1,1)
    split_coef = np.split(coef,coef_split_idx)
    eta_mu = Xs[0]@split_coef[0]
    eta_sd = Xs[1]@split_coef[1]

    mu = links[0].fi(eta_mu) # vector of means
    sd = links[1].fi(eta_sd) # vector of scales

    # Your code goes here. 
    pass

You could now estimate the model - based only on the log-likelihood function that you provided. However, that would take quite some time, because we then have to approximate the gradient of the log-likelihood. I thus provided the analytic solution for the gradient computation in the ``LBFGSGENSMOOTHFamily`` implementation and initialize the model from a GAMM fit. This speeds up estimation (see code below) quite a bit.

In [ ]:
formula_mu = Formula(lhs("rts_var"),[i(),l(["conditions"]),f(["logGoogleFreq"],by="conditions"),fs(["logGoogleFreq"],rf="subjects_var")],data=dat)
formula_sd = Formula(lhs("rts_var"),[i()],data=dat) # phi is constant for all Rvs!

# Collect both formulas
formulas = [formula_mu,formula_sd]
links = [Identity(),LOG()]

# init coef for gsmm like for gammlss
mean_model = GAMM(copy.deepcopy(formula_mu),family=Gaussian())
mean_model.fit(progress_bar=False)
m_coef,_ = mean_model.get_pars()

init_coef = np.concatenate((m_coef.reshape(-1,1),
                            np.ones((formula_sd.n_coef)).reshape(-1,1)))

# Now define the general family + model and fit!
gsmm_fam = LBFGSGENSMOOTHFamily(2,links,llk_gamm_fun)
gsmm = GSMM(formulas=formulas,family=gsmm_fam)

gsmm.fit(init_coef=init_coef,optimizer="L-BFGS-B",extend_lambda=True,use_grad=True,
          seed=20,maxcor=30,progress_bar=True)

Plot the estimated effects - compare those to the effects estimated for model 1, what do you notice?

In [ ]:
plot(gsmm)

## Exercise 8

As mentioned, we can now model $\phi$ (in this case $\sigma$) as well. Update ``formula_sd`` to also include a smooth of ``logGoogleFreq``, then refit & plot the resulting model ``gsmm2`` (hint: set the ``dist_par`` argument to 1 when calling ``plot(gsmm2)`` to visualize the estimated smooth for $\phi$).

In [ ]:
# Your code goes here.
formula_sd2 = None